# 1. Business Understanding

Listing any type of property in AirBnb has become an important source of income for many owners, and also is an affordable acommodation option for travelers in a budget (although you can also find [castles](https://www.airbnb.com.co/rooms/9859896?_set_bev_on_new_domain=1614045046_YWZlODRlMTIwNzZi&source_impression_id=p3_1614045065_a6TPR4xPl0MoDwRs&guests=1&adults=1) and [islands](https://www.airbnb.com.co/rooms/1296802?_set_bev_on_new_domain=1614045046_YWZlODRlMTIwNzZi&source_impression_id=p3_1614045210_eUMTMwo0wunP4gz1))


## Boston?

The real reason behind the selection of Boston as an Airbnb study case is purely based in the Udacity Nanodregree recomendation, but there are some interesting fats surrounding the listings in this city.

- A 2016 study suggested that a 10% increase in Airbnb listings led to a 0.42% increase in rent prices being asked in that area. [fact 1](https://ipropertymanagement.com/research/airbnb-statistics)
- Boston is the 12th highest occupancy rate city in the U.S. [fact 2](https://www.stratosjets.com/blog/airbnb-statistics/)
- "...The new rules require that hosts own the properties they rent out, and live in them for at least nine months of the year." [fact 3](https://www.cnbc.com/2019/12/03/in-boston-airbnb-tasked-with-removing-thousands-of-illegal-listings.html)


## What data do we have?

The dataset has three archives to work with: 

- ***Listings***, including full descriptions and average review score
- ***Reviews***, including unique id for each reviewer and detailed comments
- ***Calendar***, including listing id and the price and availability for that day

## What question do we want to answer?

1. Is there is a strong correlation between the size of the listing and its price?
2. Is Location the most important variable for demand and pricing?
3. Past reviews impact future listings of the place?

# 2. Data Understanding

###  Environment setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns

cal = pd.read_csv('./calendar.csv')
lis = pd.read_csv('./listings.csv')
rev = pd.read_csv('./reviews.csv')


In [2]:
cal.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


# 3. Data Preparation

# 4. Modeling

# 5. Results, Evaluation and Insights